# Part 2: RAG


## 1. Import

In [1]:
from typing import Literal, Any
from copy import deepcopy

from typing_extensions import TypedDict
import matplotlib.pyplot as plt
import numpy as np
import random
from decouple import config
from pydantic import BaseModel, Field
from IPython.display import Image, display
from tqdm import tqdm
from dotenv import load_dotenv

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain.prompts import PromptTemplate
from langchain_ibm import WatsonxEmbeddings
from langchain_ibm import WatsonxLLM
from langgraph.graph import START, StateGraph
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from langchain_text_splitters import RecursiveCharacterTextSplitter

import litellm
from litellm import completion
import instructor
from instructor import Mode

import pandas as pd
import os
import re

## 2. Load API key and model setup

In [2]:
print(os.path.exists(".env")) 

True


In [3]:
# Load .env file
load_dotenv() 

# Get the API key
WX_API_KEY = os.getenv('WX_API_KEY')
WX_PROJECT_ID = os.getenv('WX_Project_ID')

if WX_API_KEY:
    print("API Key Loaded Successfully")
else:
    print("API Key Not Found! Check your .env file.")

WX_API_URL = "https://us-south.ml.cloud.ibm.com"


API Key Loaded Successfully


In [4]:
llm = WatsonxLLM(

        model_id= "ibm/granite-3-8b-instruct",
        url=WX_API_URL,
        apikey=WX_API_KEY,
        project_id=WX_PROJECT_ID,

        params={
            GenParams.DECODING_METHOD: "greedy",
            GenParams.TEMPERATURE: 0,
            GenParams.MIN_NEW_TOKENS: 5,
            GenParams.MAX_NEW_TOKENS: 1_000,
            GenParams.REPETITION_PENALTY:1.2
        }

)

In [5]:
llm_result = llm.generate(["Hi how are you?"])

print(type(llm_result))
print(llm_result)

<class 'langchain_core.outputs.llm_result.LLMResult'>
generations=[[Generation(text="\nI'm an artificial intelligence and don't have feelings, but I'm here to help you. How can I assist you today?", generation_info={'finish_reason': 'eos_token'})]] llm_output={'token_usage': {'generated_token_count': 31, 'input_token_count': 5}, 'model_id': 'ibm/granite-3-8b-instruct', 'deployment_id': None} run=[RunInfo(run_id=UUID('fb0b6db9-a24c-4c49-9891-2c694a7c3c52'))] type='LLMResult'


## 3. Load the markdown file

In [6]:

document = TextLoader(r"C:\Users\charl\Documents\CBS\Code Projects_Python\AIML25\AIML25_Project\parsed_markdown\consolidated_markdown_data.md").load()[0]
document.metadata

{'source': 'C:\\Users\\charl\\Documents\\CBS\\Code Projects_Python\\AIML25\\AIML25_Project\\parsed_markdown\\consolidated_markdown_data.md'}

## 4. Chunking the data from the markdown

In [7]:
#find product headers in Header 2 (##) and promote to Header 1 (#)
def normalize_markdown_headers(text):
    lines = text.split('\n') #splitting the text by each line
    normalized_lines = []
    for line in lines:
        if re.match(r'#{2,3}\s+K\s*\d+.*', line):  #searches for a format of the product name (##\s+ = header 2, K\s*\d+ = the letter K followed by a space and at least 1 digit)
            line = '# ' + line.lstrip('#').strip()  #promotes the found line to Header 1
        normalized_lines.append(line)
    return '\n'.join(normalized_lines)


In [8]:
normalized_content = normalize_markdown_headers(document.page_content)

#splitting the markdown file based on headers
headers_to_split_on = [("#", "Header 1"),("##", "Header 2"), ("###", "Header 3")]
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
header_chunks = markdown_splitter.split_text(normalized_content)

#each header chunk is then split again by token size
token_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

#applying the token-based split by looping through each header chunk
final_chunks = []
for chunk in header_chunks:
    sub_chunks = token_splitter.split_text(chunk.page_content)
    for sub_chunk in sub_chunks:
        final_chunks.append(Document(page_content=sub_chunk, metadata=chunk.metadata))



In [9]:
#filtering out content under the header "page footer"/"page header"/"page number" or data that is mostly links
filtered_chunks = []
for chunk in final_chunks:
    is_footer = (
    chunk.metadata.get("Header 2") == 'Page Header' or
    chunk.metadata.get("Header 2") == "Page Footer" or
    chunk.metadata.get("Header 2") == "Page Number"
)
    

    if is_footer:
        continue  
    filtered_chunks.append(chunk)


In [10]:
#testing to find a value in the chunks
for i, chunk in enumerate(filtered_chunks):
    if "574,99 €" in chunk.page_content:
        print(f"FOUND in chunk {i}:\n{chunk.page_content}\n")

FOUND in chunk 1984:
574,99 € <!-- text, from page 0 (l=0.089,t=0.882,r=0.251,b=0.907), with ID d921a42c-dd4f-4fd7-99a0-2d2839878787 -->

FOUND in chunk 2200:
- **Price**: 574,99 €
- **Includes**: MwSt. (Mehrwertsteuer)
- **Delivery**: kostenlose Lieferung ab 50 € <!-- text, from page 0 (l=0.087,t=0.882,r=0.404,b=0.932), with ID 33768bdd-fdd2-4b49-a931-9860c75eebb6 -->



In [11]:
filtered_chunks [:20]

[Document(metadata={}, page_content='<!-- ===== K 2 Battery _ Kärcher.md ===== -->'),
 Document(metadata={'Header 1': 'K 2 BATTERY'}, page_content='Der leistungsstarke Akku-Hochdruckreiniger K 2 Battery für vielseitige, flexible Einsätze ohne Stromanschluss. 36-V-Wechselakku und Ladegerät sind als Sonderzubehör separat erhältlich. <!-- text, from page 0 (l=0.064,t=0.080,r=0.932,b=0.148), with ID 329efb5d-cc5e-4ef2-8d0a-9f47c52857eb -->'),
 Document(metadata={'Header 1': 'K 2 BATTERY', 'Header 3': 'Description'}, page_content='The image shows a yellow and black pressure washer from the brand Kärcher. It is a compact, portable model with a handle at the top for easy carrying. The device features a battery compartment at the top, indicating it is battery-operated. The Kärcher logo is prominently displayed on the front.  \n#### Key Features:  \n- **Color**: Predominantly yellow with black accents.\n- **Design**:\n- The pressure washer has a sturdy handle for portability.'),
 Document(metad

In [12]:
#adding the header prefic to page content
def update_documents_with_headers(chunks):
    
    updated_chunks = []
    max_depth=3  #header 1,2,3
    
    for doc in chunks:
        new_doc = deepcopy(doc)  #creating a deep copy of the document to not modify the original
        
        #get all headers from the metadata
        headers = []
        for i in range(1, max_depth + 1):
            key = f'Header {i}'
            if key in new_doc.metadata:
                headers.append(f"{key}: {new_doc.metadata[key]}")
        
        #creating the header prefix and update page_content
        if headers:
            prefix = f"[{'/'.join(headers)}]: "
            new_doc.page_content = prefix + "\n" + new_doc.page_content
        
        updated_chunks.append(new_doc)
    
    return updated_chunks

#applying this function to the filtered chunks
docs = update_documents_with_headers(filtered_chunks)

In [13]:
#clean the data from urls and sequences of whitespaces and changing the writing, so that only the first letter of each word is capital
def clean_content(text):


    text = re.sub(r"http\S+", "", text)  
    text = re.sub(r"\s+", " ", text).strip()
    text = text.title()

    return text

In [14]:
# apply clean_content to each document's page_content
for doc in docs:
    doc.page_content = clean_content(doc.page_content)

In [15]:
for doc in docs[:10]:
    print(doc.page_content, end="\n\n")

<!-- ===== K 2 Battery _ Kärcher.Md ===== -->

[Header 1: K 2 Battery]: Der Leistungsstarke Akku-Hochdruckreiniger K 2 Battery Für Vielseitige, Flexible Einsätze Ohne Stromanschluss. 36-V-Wechselakku Und Ladegerät Sind Als Sonderzubehör Separat Erhältlich. <!-- Text, From Page 0 (L=0.064,T=0.080,R=0.932,B=0.148), With Id 329Efb5D-Cc5E-4Ef2-8D0A-9F47C52857Eb -->

[Header 1: K 2 Battery/Header 3: Description]: The Image Shows A Yellow And Black Pressure Washer From The Brand Kärcher. It Is A Compact, Portable Model With A Handle At The Top For Easy Carrying. The Device Features A Battery Compartment At The Top, Indicating It Is Battery-Operated. The Kärcher Logo Is Prominently Displayed On The Front. #### Key Features: - **Color**: Predominantly Yellow With Black Accents. - **Design**: - The Pressure Washer Has A Sturdy Handle For Portability.

[Header 1: K 2 Battery/Header 3: Description]: - A Battery Compartment Is Visible At The Top, Suggesting It Is Cordless. - The Front Panel Includ

## 5. Embedding the chunks

In [16]:
embed_params = {}

watsonx_embedding = WatsonxEmbeddings(
    model_id="ibm/granite-embedding-278m-multilingual", 
    url= WX_API_URL,
    project_id=WX_PROJECT_ID,
    apikey=WX_API_KEY,
    params=embed_params,
)

In [17]:
local_vector_db = Chroma.from_documents(
    collection_name="my_collection",
    embedding=watsonx_embedding,
    persist_directory="my_vector_db", # This will save the vector database to disk! Delete it if you want to start fresh.
    documents=docs,    #these are the chunks 
    
)

## 6. Vector base and  similarity search

In [18]:
# Use the vectorstore as a retriever
retriever = local_vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={
        "k": 3,  #gives out the top 3 findings
    }
)

In [19]:
retrieved_documents = retriever.invoke("What is the price in € of the K 3 Horizontal Plus Home?")

for document in retrieved_documents:
    print(f"{'#' * 80}\nID: {document.id}")
    first_n_of_content = document.page_content[:500].replace('\n\n', ' ')
    print(f"Content: {first_n_of_content}\n")

################################################################################
ID: de03e295-e05a-45f4-a851-dae52efc32e3
Content: [Header 1: K 3 Horizontal Plus Home/Header 2: Price Information]: - **Price**: 139,99 € - **Details**: Inkl. Mwst. - Kostenlose Lieferung Ab 50 € <!-- Key_Value, From Page 0 (L=0.089,T=0.882,R=0.404,B=0.932), With Id 6Eec2Ff0-5133-4A70-9F5A-A084Dc62E5Ca -->

################################################################################
ID: 08d8bac5-4287-4faf-99f4-8e50d996ae47
Content: [Header 1: K 3 Horizontal Plus Home | Kärcher]: This Is A Page Header Indicating The Product Name "K 3 Horizontal Plus Home" By The Brand "Kärcher". <!-- Page_Header, From Page 0 (L=0.473,T=0.019,R=0.685,B=0.028), With Id 27110F42-86E4-4F23-B0Fd-E50788F227Ca -->

################################################################################
ID: cf4838dc-9ffc-42a5-a8f9-6921a4d2d8f1
Content: [Header 1: K 3 Horizontal Plus Home | Kärcher]: This Is A Page Header Indicating The

## 7. RAG prompt template

In [20]:
few_shot_examples = """
Example 1:
Question: What is the price of the product 'K 7 Smart Control Flex Home'?
Answer: 644.99 €

Example 2:
Question: What is the delivery time of the product 'K 5 FJ Home'?
Answer: 3-4 Werktage

Example 3:
Question: What is the field 'Stromart (V/Hz)' for product '	K 4 WCM'?
Answer: 230 / 50
"""

template = """You are an assistant helping to complete missing fields in a product database by extracting accurate information from documentation.

Each question corresponds to a missing field in the product's entry (e.g., price, description, accessories). Use the retrieved context to find and return the correct value.

If the context does not contain the necessary information, respond with "Unknown". Keep answers factual, concise, and suitable for filling into a CSV cell. Do not speculate or include explanations. 

Do not answer in a sentence but just with the value.

{few_shot_examples}

Question:
{question}

Context: 
{context}

Answer:
"""
prompt = PromptTemplate.from_template(template)

## 8. RAG pipeline

In [21]:
prompt.invoke(
    input={
     "question": "What is the price in € of the product K 3 Plus Home?",
     "context": retrieved_documents[0].page_content,
     "few_shot_examples": few_shot_examples
      }
    )

StringPromptValue(text='You are an assistant helping to complete missing fields in a product database by extracting accurate information from documentation.\n\nEach question corresponds to a missing field in the product\'s entry (e.g., price, description, accessories). Use the retrieved context to find and return the correct value.\n\nIf the context does not contain the necessary information, respond with "Unknown". Keep answers factual, concise, and suitable for filling into a CSV cell. Do not speculate or include explanations. \n\nDo not answer in a sentence but just with the value.\n\n\nExample 1:\nQuestion: What is the price of the product \'K 7 Smart Control Flex Home\'?\nAnswer: 644.99 €\n\nExample 2:\nQuestion: What is the delivery time of the product \'K 5 FJ Home\'?\nAnswer: 3-4 Werktage\n\nExample 3:\nQuestion: What is the field \'Stromart (V/Hz)\' for product \'\tK 4 WCM\'?\nAnswer: 230 / 50\n\n\nQuestion:\nWhat is the price in € of the product K 3 Plus Home?\n\nContext: \n[

In [22]:
question = "What is the price in € of the product K 3 Plus Home?"

retrieved_docs = local_vector_db.similarity_search(question)
docs_content = "\n\n".join(f"Document {i+1}:\n{doc.page_content}" for i, doc in enumerate(retrieved_docs))
formated_prompt = prompt.invoke({"question": question, "context": docs_content, "few_shot_examples": few_shot_examples})

print(formated_prompt.to_string()[:1000])

You are an assistant helping to complete missing fields in a product database by extracting accurate information from documentation.

Each question corresponds to a missing field in the product's entry (e.g., price, description, accessories). Use the retrieved context to find and return the correct value.

If the context does not contain the necessary information, respond with "Unknown". Keep answers factual, concise, and suitable for filling into a CSV cell. Do not speculate or include explanations. 

Do not answer in a sentence but just with the value.


Example 1:
Question: What is the price of the product 'K 7 Smart Control Flex Home'?
Answer: 644.99 €

Example 2:
Question: What is the delivery time of the product 'K 5 FJ Home'?
Answer: 3-4 Werktage

Example 3:
Question: What is the field 'Stromart (V/Hz)' for product '	K 4 WCM'?
Answer: 230 / 50


Question:
What is the price in € of the product K 3 Plus Home?

Context: 
Document 1:
[Header 1: K 3 Horizontal Plus Home/Header 2: Pri

In [23]:
answer = llm.invoke(formated_prompt)

print(answer)

The price of the product 'K 3 Plus Home' is 139.99 €.


### 8.1 RAG pipeline with LangGraph

In [24]:
# Define state for application
class State(TypedDict):
    """ A langgraph state for the application """
    question: str
    context: list[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    """ Our retrieval step. We use our local vector database to retrieve similar documents to the question """
    retrieved_docs = local_vector_db.similarity_search(state["question"], k=3) 
    return {"context": retrieved_docs} 


def generate(state: State):
    """ Our generation step. We use the retrieved documents to generate an answer to the question """

    # Format the prompt
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    formated_prompt = prompt.invoke({"question": state["question"], "context": docs_content, "few_shot_examples": few_shot_examples})

    # Generate the answer
    response = llm.invoke(formated_prompt)
    return {"answer": response}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve") # Start at the retrieve step
graph = graph_builder.compile() # Compile the graph

In [25]:
response = graph.invoke({"question": "What is the price in € of the product K 7 Premium Power Flex?"})

response

{'question': 'What is the price in € of the product K 7 Premium Power Flex?',
 'context': [Document(id='7a450759-c87f-41dc-a7b0-ba89b28a697e', metadata={'Header 1': 'K 7 POWER FLEX HOME <!-- title, from page 0 (l=0.065,t=0.080,r=0.436,b=0.100), with ID dea94208-49e1-4a22-82f3-a1199f26c167 -->', 'Header 2': 'Price'}, page_content='[Header 1: K 7 Power Flex Home <!-- Title, From Page 0 (L=0.065,T=0.080,R=0.436,B=0.100), With Id Dea94208-49E1-4A22-82F3-A1199F26C167 -->/Header 2: Price]: 574,99 € <!-- Text, From Page 0 (L=0.089,T=0.882,R=0.251,B=0.907), With Id D921A42C-Dd4F-4Fd7-99A0-2D2839878787 -->'),
  Document(id='3432a15a-a552-4e04-9aa7-a0d23d24edc8', metadata={'Header 1': 'K 7 PREMIUM POWER FLEX HOME', 'Header 2': 'Price Information'}, page_content='[Header 1: K 7 Premium Power Flex Home/Header 2: Price Information]: - **Price**: 624,99 € - **Includes**: Mwst. (Vat) - **Delivery**: Kostenlose Lieferung Ab 50 € (Free Delivery From 50 €) <!-- Key_Value, From Page 0 (L=0.087,T=0.882,R=

## 9. Evaluation

In [26]:
df_edited = pd.read_csv(r"C:\Users\charl\Documents\CBS\Code Projects_Python\AIML25\AIML25_Project\data\SAP_Produktstammdaten_vfinal.csv", sep=",")

### 9.1 Deleting random data from the csv

In [43]:
#columns to exclude from deletion
protected_columns = ["Produktname", "Bestellnummer"]

#identify non-empty cells excluding the protected column
non_empty_cells = [
    (i, col)
    for i in df_edited.index
    for col in df_edited.columns
    if col not in protected_columns and pd.notna(df_edited.at[i, col])
]

#define how many values to drop 
drop_fraction = 0.03  #3%
num_to_drop = int(len(non_empty_cells) * drop_fraction)

#randomly selects which cells to drop
cells_to_drop = random.sample(non_empty_cells, num_to_drop)

#setting selected cells to NaN
for i, col in cells_to_drop:
    df_edited.at[i, col] = np.nan


In [44]:
missing_fields = []  #list of deleted/mising cells

#looping through the csv file based on the column "product name" and adding all NaN or "" cells to the list
for idx, row in df_edited.iterrows():
    product_name = row["Produktname"]
    
    for field in df_edited.columns:
        value = row[field]
        if pd.isna(value) or str(value).strip().lower() in [""]:
            missing_fields.append((idx, product_name, field))

#display all missing values
for idx, product_name, field in missing_fields:
    print(f"Row {idx} – {product_name} is missing: {field}")


Row 0 – K 7 Premium Smart Control Flex eco!B is missing: Lieferzeit
Row 0 – K 7 Premium Smart Control Flex eco!B is missing: Anschlussleistung (kW)
Row 0 – K 7 Premium Smart Control Flex eco!B is missing: Lieferumfang
Row 1 – K 7 Premium Smart Control Flex Home is missing: Anschlussleistung (kW)
Row 3 – K 7 Premium Power Flex Home is missing: Fördermenge (l/h)
Row 3 – K 7 Premium Power Flex Home is missing: Flächenleistung (m²/h)
Row 5 – K 7 Smart Control Flex eco!Booster is missing: Gewicht inkl. Verpackung (kg)
Row 9 – K 7 Smart Control Flex is missing: Preis (€ inkl. MwSt.)
Row 10 – K 7 WCM Premium Home is missing: Stromart (V/Hz)
Row 10 – K 7 WCM Premium Home is missing: Abmessungen (L × B × H) (mm)
Row 11 – K 7 WCM is missing: Lieferzeit
Row 11 – K 7 WCM is missing: Farbe
Row 12 – K 7 WCM Car&Home is missing: Lieferzeit
Row 12 – K 7 WCM Car&Home is missing: Stromart (V/Hz)
Row 12 – K 7 WCM Car&Home is missing: Gewicht inkl. Verpackung (kg)
Row 13 – K 7 WCM FJ is missing: Anschluss

### 9.2 Accuracy rating

In [45]:
#loading the csv file again to use it as comparison 
df_original= pd.read_csv(r"C:\Users\charl\Documents\CBS\Code Projects_Python\AIML25\AIML25_Project\data\SAP_Produktstammdaten_vfinal.csv", sep=",")

In [46]:
def normalize_answer(answer):
    #remove unnecessary words and units 
        normalized_answer = re.sub(r"(kg|kW|L/h|bar|MPa|M²/H|°C|cm|mm)", "", answer)
    #remove extra spaces, newlines, or any text that isn't the core numeric value but keep ".", "/", "-"
        normalized_answer = re.sub(r"[^a-zA-Z0-9\s./-äöü-]", "", normalized_answer)
    #remove "\n" and everything after
        normalized_answer = re.sub(r'\n.*', ' ', normalized_answer)
    #normalize the format to lowercase
        normalized_answer = normalized_answer.lower()
        
        return normalized_answer

results = []

for idx, product_name, field in missing_fields:
    question = f"What is the {field} for product '{product_name}'?"

    #running the LangGraph application
    state_input = {"question": question}
    output_state = graph.invoke(state_input)

    answer = output_state["answer"]

    answer= normalize_answer(answer)

    #define ground truth from the original DataFrame
    try:
        ground_truth_value = df_original.at[idx, field]
    except KeyError:
        ground_truth_value = None

    is_correct = (
        str(answer).strip().lower() == str(ground_truth_value).strip().lower()
        if pd.notna(ground_truth_value) else None
    )

    results.append({
        "index": idx,
        "product": product_name,
        "field": field,
        "answer": answer,
        "ground_truth": ground_truth_value,
        "correct": is_correct,
    })


In [47]:
#showing some results
results[:20]

[{'index': 0,
  'product': 'K 7 Premium Smart Control Flex eco!B',
  'field': 'Lieferzeit',
  'answer': '2-3 werktagen',
  'ground_truth': '2-3 Werktage',
  'correct': False},
 {'index': 0,
  'product': 'K 7 Premium Smart Control Flex eco!B',
  'field': 'Anschlussleistung (kW)',
  'answer': 'the anschlussleistung  for product k 7 premium smart control flex ecob is 3 .',
  'ground_truth': '3',
  'correct': False},
 {'index': 0,
  'product': 'K 7 Premium Smart Control Flex eco!B',
  'field': 'Lieferumfang',
  'answer': 'inklusive mwst. kostenloser versand ab 50 ',
  'ground_truth': 'G 180 Q Smart Control, Multi Jet 3-in-1, eco!Booster, HD-Schlauch 10 m PremiumFlex, Adapter Gartenschlauchanschluss A3/4',
  'correct': False},
 {'index': 1,
  'product': 'K 7 Premium Smart Control Flex Home',
  'field': 'Anschlussleistung (kW)',
  'answer': 'the anschlussleistung  for product k 7 premium smart control flex home is 3 .',
  'ground_truth': '3',
  'correct': False},
 {'index': 3,
  'product': '

In [48]:
# Convert results to DataFrame
eval_df = pd.DataFrame(results)

# Accuracy
if "correct" in eval_df.columns:
    accuracy = eval_df["correct"].dropna().mean()
    print(f"LLM Accuracy: {accuracy:.2%}")


LLM Accuracy: 23.73%


In [49]:
column_accuracy = eval_df.dropna(subset=["correct"]).groupby("field")["correct"].mean()

print("\nAccuracy per column:")
print(column_accuracy)


Accuracy per column:
field
Abmessungen (L × B × H) (mm)     0.20
Anschlusskabel (m)               1.00
Anschlussleistung (kW)           0.00
Ausstattung                      0.00
Druck (bar/MPa)                  0.00
Farbe                            0.00
Flächenleistung (m²/h)           0.00
Fördermenge (l/h)                0.80
Gewicht inkl. Verpackung (kg)    0.25
Gewicht ohne Zubehör (kg)        0.00
Lieferumfang                     0.00
Lieferzeit                       0.20
Preis (€ inkl. MwSt.)            0.00
Stromart (V/Hz)                  0.60
Zulauftemperatur (°C)            0.00
Name: correct, dtype: float64
